In [4]:
import numpy as np
from numpy import exp, sqrt, pi

In [5]:
def flip_at_index(binary_string, index):
    """
    Flip the character at the specified index in a binary string.
    
    :param binary_string: A string consisting of '0's and '1's.
    :param index: The index of the character to flip.
    :return: The modified string with the character at the given index flipped.
    """
    if index < 0 or index >= len(binary_string):
        raise IndexError("Index out of range")

    # Convert the string to a list
    char_list = list(binary_string)

    # Flip the character at the specified index
    char_list[index] = '0' if char_list[index] == '1' else '1'

    # Convert the list back to a string and return
    return ''.join(char_list)

def replace_at_index(binary_string, index):
    """
    Replace the character at the specified index in a binary string with '0' and '1'.

    :param binary_string: A string consisting of '0's and '1's.
    :param index: The index of the character to replace.
    :return: Two strings, one with the character at the index replaced by '0' and the other by '1'.
    """
    if index < 0 or index >= len(binary_string):
        raise IndexError("Index out of range")

    # Convert the string to a list
    char_list = list(binary_string)

    # Create two copies of the list
    list_with_0 = char_list.copy()
    list_with_1 = char_list.copy()

    # Replace the character at the specified index
    list_with_0[index] = '0'
    list_with_1[index] = '1'

    # Convert the lists back to strings
    return ''.join(list_with_0), ''.join(list_with_1)

def round_complex(z):
    """
    Round the real and imaginary parts of a complex number to five significant figures.

    :param z: Complex number to be rounded.
    :return: Complex number with real and imaginary parts rounded to five significant figures.
    """
    return complex(round(z.real, 5), round(z.imag, 5))

# QuantumSimulator class that create a quantum circuit and simulate the effect of the quantum gates
# state vector is stored in dict, where the key correspond to the state, and the values correspond to probability amplittude
class QuantumSimulator:
    def __init__(self, n_qubits, n_cqbits):
        """
        Initialize the attributes to describe a QuantumSimulator with number of qubits.
        """
        initial_state = '0' * n_qubits
        state_vector = {initial_state : 1}
        self.state = state_vector
        self.cRegisters = ['None'] * n_cqbits
    
    # apply x gate on the target 
    def x(self, target):
        new_state = {}
        for key, value in self.state.items():
            new_key, new_value = key, value
            new_key = flip_at_index(new_key, target)
            new_state[new_key] = new_value
        self.state = new_state

    # apply h gate on the target
    def h(self, target):
        new_state = {}
        for key, value in self.state.items():
            factor = 1
            if key[target] == '1':
                factor = -1
            new_key1, new_key2 = replace_at_index(key, target)
            new_state[new_key1] = round_complex(1/sqrt(2)*value + new_state.get(new_key1, complex(0)))
            new_state[new_key2] = round_complex(factor*1/sqrt(2)*value + new_state.get(new_key2, complex(0)))
        self.state = new_state
    
    # apply t gate on the target
    def t(self, target):
        new_state = {}
        for key, value in self.state.items():
            new_key, new_value = key, value
            if new_key[target] == '1':
                new_value = round_complex(np.exp(1j*pi/4)*new_value)
            new_state[new_key] = new_value
        self.state = new_state      
    
    # apply tdg gate on the target
    def t_dag(self, target):
        new_state = {}
        for key, value in self.state.items():
            new_key, new_value = key, value
            if new_key[target] == '1':
                new_value = round_complex(np.exp(-1j*pi/4)*new_value)
            new_state[new_key] = new_value
        self.state = new_state  

    # apply cx gate based on the control on the target
    def cx(self, control, target):
        new_state = {}
        for key, value in self.state.items():
            new_key, new_value = key, value
            if new_key[control] == '1':
                new_key = flip_at_index(new_key, target)
            new_state[new_key] = new_value
        self.state = new_state
    
    def measure(self, q_index, c_index):
        return 0
    
    # clean up the state with 0 probability
    def clean(self):
        new_state = {}
        for key, value in self.state.items():
            new_key, new_value = key, value
            if np.abs(value) != 0:
                new_state[new_key] = new_value
        self.state = new_state
    
    # clean the state and return state_vector output
    def state_out(self):
        self.clean()
        return self.state
    

In [9]:
qc = QuantumSimulator(2, 4)
qc.h(0)
qc.cx(0, 1)
qc.t_dag(1)
qc.state_out()

{'00': (0.70711+0j), '11': (0.5-0.5j)}